# Backward Chaining

This Jupyter notebook demonstrates how *Backward Chaining* can be used to infere knowledge from a First-Order Logic knowledge base.

Import modules:

In [1]:
from utils import *
from logic import *
from notebook import psource, Canvas_fol_bc_ask

### Initialize Knowledge Base:

We consider the *Criminal West* example from the lecture. The example makes use of the following predicate symbols:

* `Criminal(x)`: `x` is a criminal
* `American(x)`: `x` is an American
* `Enemy(x ,y)`: `x` is an enemy of `y`
* `Sells(x ,y, z)`: `x` sells `y` to `z`
* `Owns(x ,y)`: `x` owns `y`
* `Weapon(x)`: `x` is a weapon
* `Hostile(x)`: `x` is a hostile nation

Furthermore, the constant symbols `Nono`, `West`, and `America` are given.

Before we can apply the *Forward Chaining* algorithm, we have to construct our knowledge base by adding all relevant clauses:

In [2]:
clauses = []

<em>“... it is a crime for an American to sell weapons to hostile nations”</em><br/>

$\text{American}(x) \land \text{Weapon}(y) \land \text{Sells}(x, y, z) \land \text{Hostile}(z) \implies \text{Criminal} (x)$

In [3]:
clauses.append(expr("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))

<em>"Nono ... has some missiles"</em><br/>
This states the existence of some missile which is owned by Nono. $\exists x \text{Owns}(\text{Nono}, x) \land \text{Missile}(x)$. We invoke existential instantiation to introduce a new constant `M1` which is the missile owned by Nono.

$\text{Owns}(\text{Nono}, \text{M1}), \text{Missile}(\text{M1})$

In [4]:
clauses.append(expr("Owns(Nono, M1)"))
clauses.append(expr("Missile(M1)"))

<em>".. all of its missiles were sold to it by Colonel West"</em><br/>

$\text{Missile}(x) \land \text{Owns}(\text{Nono}, x) \implies \text{Sells}(\text{West}, x, \text{Nono})$

In [5]:
clauses.append(expr("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))

<em>“Missiles are weapons”</em><br/>

$\text{Missile}(x) \implies \text{Weapon} (x)$

In [6]:
clauses.append(expr("Missile(x) ==> Weapon(x)"))

<em>“An enemy of America counts as hostile”</em><br/>

$\text{Enemy}(x, \text{America}) \implies \text{Hostile}(x)$

In [7]:
clauses.append(expr("Enemy(x, America) ==> Hostile(x)"))

<em>"West, who is American ..."</em><br/>

$\text{American}(\text{West})$

In [8]:
clauses.append(expr("American(West)"))

<em>"The country Nono, an enemy of America ..."</em><br/>

$\text{Enemy}(\text{Nono}, \text{America})$

In [9]:
clauses.append(expr("Enemy(Nono, America)"))

Now that we have converted the information into first-order definite clauses we can create our first-order logic knowledge base.

In [10]:
crime_kb = FolKB(clauses)

### Backward Chaining Algorithm:

Now we want to check if Colonel West is a criminal by applying the *Backward Chaining* algorithm, which is implemented in the `fol_bc_ask` function:

In [11]:
psource(fol_bc_ask)

The `fol_bc_ask` function internally call the `fol_bc_or` function which select any clause of the form $\text{lhs} \implies \text{goal}$. Looking at all rules's `lhs` whose `rhs` unify with the `goal`, we yield a substitution which proves all the conjuncts in the `lhs`. We use `parse_definite_clause` to attain `lhs` and `rhs` from a clause of the form $\text{lhs} \implies \text{rhs}$. For atomic facts the `lhs` is an empty list.

In [12]:
psource(fol_bc_or)

The `fol_bc_or` function then calls the `fol_fc_and` function to prove all the conjunctions in the `lhs`. We need to find a substitution which proves each <em>and</em> every clause in the list of conjuncts.

In [13]:
psource(fol_bc_and)

To check if Colonel West is a criminal we apply the *Backward Chaining* algorithm for the query $\alpha = \text{Criminal(West)}$:

In [14]:
canvas_bc_ask = Canvas_fol_bc_ask('canvas_bc_ask', crime_kb, expr('Criminal(West)'))